<a href="https://colab.research.google.com/github/bhaskarsaripella4/AgenticAI_python/blob/main/off_label_drug_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import time
import pandas as pd
import requests
import json
import google.generativeai as genai

In [17]:
database_path = {path}
gemini_key = {key}

In [20]:
REQUEST_INTERVAL_SECONDS = 0
# 🔑 Replace with your Gemini API key
GEMINI_API_KEY = gemini_key
GEMINI_URL = f"https://generativelanguage.googleapis.com/v1/models/gemma-3-1b-it:generateContent?key={GEMINI_API_KEY}"
GEMINI_MODEL = 'gemma-3-1b-it'

In [4]:
drug_df = pd.read_csv(database_path, encoding='ISO-8859-1')
drug_df = drug_df.loc[:, ~drug_df.columns.str.startswith('Unnamed')]
print(drug_df.head())

          Drug Name      Drug Composition                 Classification  \
0           Aspirin  Acetylsalicylic acid                          NSAID   
1          Naproxen              Naproxen                          NSAID   
2       Paracetamol         Acetaminophen  NSAID (Analgesic/Antipyretic)   
3         Ibuprofen             Ibuprofen                          NSAID   
4  Benzoyl Peroxide      Benzoyl Peroxide        Topical Acne Medication   

                                     Chemical Name Chemical Formula  \
0                          2-acetyloxybenzoic acid              NaN   
1  (2S)-2-(6-methoxynaphthalen-2-yl)propanoic acid              NaN   
2                    N-(4-hydroxyphenyl) acetamide          C8H9NO2   
3     2-[4-(2-methylpropyl) phenyl] propanoic acid              NaN   
4                                 Benzoyl Peroxide         C14H10O4   

                                    Recommended Uses  \
0  Angina|Heart attacks|Strokes|Pain relief|Fever...   
1  A

In [11]:
#test case for model whether it works or not

from google import genai
from google.genai import types

client = genai.Client(
    api_key=gemini_key,
    http_options=types.HttpOptions(api_version='v1alpha')
)

response = client.models.generate_content(
    # model='gemini-2.0-flash-001', contents='Why is the sky blue?'
    model = 'gemma-3-1b-it', contents = ["hydrocortisone","skin rash in a patient with ulcerative colitis"]
)
print(response.text)

Okay, let's break down hydrocortisone skin rash in a patient with ulcerative colitis. This is a relatively common and concerning presentation, and it requires careful investigation. Here's a detailed look at the potential causes, diagnostic approaches, and management strategies:

**1. Understanding the Situation – Why Hydrocortisone?**

* **Hydrocortisone's Role:** Hydrocortisone is a synthetic glucocorticoid, a powerful anti-inflammatory medication. It's frequently prescribed to manage inflammation in ulcerative colitis, particularly in flares.  It's used to reduce redness, swelling, and pain.
* **Why it's a Concern in UC:**  While hydrocortisone can be helpful, it can also cause a significant skin reaction in individuals with ulcerative colitis. The inflammation in the gut and the skin can interact, leading to a paradoxical effect – the medication exacerbating the skin issue.

**2. Possible Causes of Hydrocortisone Rash in a UC Patient**

This is where it gets nuanced. There isn't on

In [5]:
import google.generativeai as genai

genai.configure(api_key=gemini_key)

# List available models
for m in genai.list_models():
    print(m.name, " - ", m.supported_generation_methods)

models/embedding-gecko-001  -  ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest  -  ['generateContent', 'countTokens']
models/gemini-pro-vision  -  ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest  -  ['generateContent', 'countTokens']
models/gemini-1.5-pro-002  -  ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro  -  ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest  -  ['generateContent', 'countTokens']
models/gemini-1.5-flash  -  ['generateContent', 'countTokens']
models/gemini-1.5-flash-002  -  ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b  -  ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-001  -  ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-latest  -  ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-2.5-pro-preview-03-25  -  ['generateContent', 'countTokens', 'cre

In [22]:
def find_chemical_names(prompt):
    prompt = prompt.lower()
    matched_drugs = []

    for _, row in drug_df.iterrows():
        if pd.isna(row['Recommended Uses']):
            continue
        uses = row['Recommended Uses'].lower()
        if any(term in uses for term in prompt.split()):
            matched_drugs.append({
                'Drug': row['Drug Name'],
                'Chemical Name': row['Chemical Name'],
                'Uses': row['Recommended Uses']
            })
    return matched_drugs


def query_gemini(chemical_name, prompt):
  # url based method. Doesn't work as planned.
    headers = {"Content-Type": "application/json"}
    payload = {
        "contents": [{
            "parts": [{
                "text": f"A patient presents with the following clinical scenario: {prompt}.\n"
                        f"Considering the drug with chemical name: {chemical_name}, "
                        f"can you suggest any possible off-label uses or alternative uses? "
                        f"Answer briefly and clinically, as a pharmacologist would."
            }]
        }]
    }

    response = requests.post(GEMINI_URL, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        return response.json()['candidates'][0]['content']['parts'][0]['text']
    else:
        return f"❌ Gemini API error {response.status_code}: {response.text}"


def suggest_off_label(prompt):
    matches = find_chemical_names(prompt)
    if not matches:
        return "No drug matches found in local database."

    results = []
    for idx, match in enumerate(matches):
        suggestion = query_gemini_sdk(match['Chemical Name'], prompt)
        results.append({
            "Drug": match['Drug'],
            "Chemical Name": match['Chemical Name'],
            "Approved Uses": match['Uses'],
            "Gemini Suggestion": suggestion
        })

        # # Wait before making the next request
        # if idx < len(matches) - 1:  # Don't sleep after the last one
        #     print(f"⏳ Waiting {REQUEST_INTERVAL_SECONDS} seconds to respect rate limit...")
        #     time.sleep(REQUEST_INTERVAL_SECONDS)
    return results


In [15]:
from google import genai
from google.genai import types

# Configure the client
client = genai.Client(
    api_key=gemini_key,
    http_options=types.HttpOptions(api_version='v1alpha')  # Use 'v1' doesn't work for some reason. 404 errors always
)

def query_gemini_sdk(chemical_name, prompt):
    clinical_prompt = (
        f"A patient presents with the following clinical scenario: {prompt}.\n"
        f"Considering the drug with chemical name: {chemical_name}, "
        f"can you suggest any possible off-label uses or alternative uses? "
        f"Answer briefly and clinically, as a pharmacologist would."
    )

    try:
      response = client.models.generate_content(
          model = GEMINI_MODEL,
          contents = [{"role":"user", "parts":[{"text":clinical_prompt}]}]
      )

      return response.candidates[0].content.parts[0].text

    except Exception as e:
      return f"❌ Gemini SDK error: {str(e)}"

In [23]:
results = suggest_off_label("skin rash in a patient with ulcerative colitis")

for result in results:
    print(f"\n💊 Drug: {result['Drug']}")
    print(f"🔬 Chemical Name: {result['Chemical Name']}")
    print(f"✅ Approved Uses: {result['Approved Uses']}")
    print(f"🧠 Off-label Suggestion:\n{result['Gemini Suggestion']}")


💊 Drug: Aspirin
🔬 Chemical Name: 2-acetyloxybenzoic acid
✅ Approved Uses: Angina|Heart attacks|Strokes|Pain relief|Fever|Inflammation
🧠 Off-label Suggestion:
Okay, let’s analyze this scenario.

**Possible Off-Label Uses & Alternative Considerations:**

2-Acetyloxybenzoic acid (also known as Acetylsalicylic acid) is primarily known for its anti-inflammatory and analgesic properties. However, its potential in this context is limited and requires careful consideration.

**Clinical Suggestion:**  While not a standard treatment, **very cautiously**, it *could* be considered in a patient with ulcerative colitis, *specifically* if there’s a documented inflammatory component to the rash.  Its anti-inflammatory effects *might* offer some benefit in managing the rash’s symptoms, but it’s unlikely to be a primary treatment.

**Alternative Uses (Highly Speculative & Requires Further Investigation):**

*   **Local Skin Barrier Support:**  Some research suggests it might stimulate keratinocyte prol